In [1]:

import pdfplumber
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import random
import spacy
import csv

In [8]:
import pdfplumber
import re
import csv
from nltk import sent_tokenize

def preprocess_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    return text

def extract_and_preprocess_text(file_path):
    preprocessed_sentences = [] 
    with pdfplumber.open(file_path) as pdf:
        num_pages = len(pdf.pages)

        for page_num in range(num_pages):
            page = pdf.pages[page_num]
            text = page.extract_text()
            sentences = sent_tokenize(text)
            
            for sentence in sentences:
                preprocessed_sentence = preprocess_text(sentence)
                preprocessed_sentences.append(preprocessed_sentence)

                print(f"Page {page_num + 1}, Sentence: {preprocessed_sentence}\n")

    return preprocessed_sentences

def save_to_csv(file_path, preprocessed_sentences):
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Page', 'Preprocessed Sentence'])  # Header

        for i, sentence in enumerate(preprocessed_sentences, start=1):
            csv_writer.writerow([i, sentence])

pdf_file_path = 'C:/Users/sriyo/Desktop/jets/NLP Bot/files/NCERT Biology.pdf'
csv_file_path = 'output.csv'

preprocessed_sentences = extract_and_preprocess_text(pdf_file_path)
save_to_csv(csv_file_path, preprocessed_sentences)


Page 1, Sentence: chapter 2 human reproduction 21 the male reproductive system as you are aware humans are sexually reproducing and 22 the female reproductive viviparous

Page 1, Sentence: the reproductive events in humans include system formation of gametes gametogenesis ie sperms in males 23 gametogenesis and ovum in females transfer of sperms into the female 24 menstrual cycle genital tract insemination and fusion of male and female gametes fertilisation leading to formation of zygote

Page 1, Sentence: this 25 fertilisation and is followed by formation and development of blastocyst implantation and its attachment to the uterine wall implantation 26 pregnancy and embryonic embryonic development gestation and delivery of the development baby parturition

Page 1, Sentence: you have learnt that these reproductive events occur after puberty

Page 1, Sentence: there are remarkable 27 parturition and lactation differences between the reproductive events in the male and in the female for e

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-e2e-qg")
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-e2e-qg")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
preprocessed_sentences[1]

'the reproductive events in humans include system formation of gametes gametogenesis i.e. sperms in males 2.3 gametogenesis and ovum in females transfer of sperms into the female 2.4 menstrual cycle genital tract insemination and fusion of male and female gametes fertilisation leading to formation of zygote.'

In [13]:
context = preprocessed_sentences[7]

inputs = tokenizer.encode("generate questions: " + context, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=100, num_beams=4, length_penalty=2.0, early_stopping=True)

generated_questions = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_questions)


How many testes are there?<sep> What glands are there?<sep>
